In [1]:
import torch
import torch.nn as nn
from torchcrf import CRF
import pandas as pd

/home/xps/anaconda3/envs/deeplearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
a = torch.rand(32, 8)
b = torch.zeros([32, 8], dtype=torch.float)
b[0, 0] = 1
loss = nn.BCEWithLogitsLoss(reduction='none')(a, b)

In [ ]:
# loss = torch.where(b != 0, loss, torch.tensor([0.]))
loss

In [33]:
loss = torch.where(b != 0, loss, loss*0.5)

In [37]:
loss.shape

torch.Size([32, 8])

In [44]:
num_tags = 5  # number of tags is 5
model = CRF(num_tags)

In [48]:
seq_length = 3  # maximum sequence length in a batch
batch_size = 2  # number of samples in the batch
emissions = torch.randn(seq_length, batch_size, num_tags) #n_seq x batch x logits
tags = torch.tensor([
  [0, 1], [2, 4], [3, 1]
], dtype=torch.long)  #nseq x batch
loss = model(emissions, tags)



In [57]:
a = torch.rand(30, 32, 142)
b = torch.zeros(30, 32, dtype=torch.long)
model = CRF(a.shape[-1])


In [58]:
model(a, b)

tensor(-4717.1206, grad_fn=<SumBackward0>)

In [5]:
list = []
name = ['train', 'test', 'dev']
for i in name:
    for label in open("/home/xps/educate/code/hust/XQA/docs/src/JointIDSF/PhoATIS/word-level" + f'/{i}/label'):
        if label.strip() not in list:
            list.append(label.strip())

In [7]:
len(list)

28

In [9]:
train = pd.read_csv("/home/xps/educate/code/hust/XQA/data/raw/ATIS/atis_intents_train.csv", header=None)


In [14]:
a = {'vietanh': 'ngu'}
a.update({'vie anh': 'ocho'})

(array(['atis_flight', 'atis_flight_time', 'atis_airfare', 'atis_aircraft',
        'atis_ground_service', 'atis_airline', 'atis_abbreviation',
        'atis_quantity'], dtype=object),
 4834)

In [15]:
test = pd.read_csv("/home/xps/educate/code/hust/XQA/data/raw/ATIS/atis_intents_test.csv", header=None)
test[0].unique(), len(test)

(array(['atis_flight', 'atis_airfare', 'atis_ground_service',
        'atis_airline', 'atis_flight_time', 'atis_quantity',
        'atis_abbreviation', 'atis_aircraft'], dtype=object),
 800)

In [7]:
lengths = {}
with open("/home/xps/educate/code/hust/XQA/data/raw/PhoATIS/word-level/train/seq.in", 'r') as f:
    for line in f.readlines():
        if len(line.strip().split(' ')) not in lengths.keys():
            lengths[len(line.strip().split(' '))] = 1
        else:
            lengths[len(line.strip().split(' '))] += 1 
        if len(line.strip().split(' ')) == 50:
            print(line.strip()) 
    

tôi muốn một chuyến bay khứ_hồi từ đồng_hới đến côn_đảo khởi_hành vào thứ tư ngày 16 tháng 6 đến côn_đảo vào buổi tối khoảng 7 giờ và trở về đồng_hới vào buổi tối ngày hôm sau khoảng 7 giờ hãng hàng_không nào có chuyến bay phù_hợp với tuyến đường đó


In [6]:
sorted(lengths, key = lambda d: lengths[d])

[47,
 33,
 50,
 37,
 1,
 38,
 2,
 36,
 34,
 31,
 30,
 29,
 28,
 32,
 25,
 27,
 26,
 3,
 24,
 23,
 4,
 22,
 21,
 5,
 6,
 20,
 19,
 7,
 17,
 18,
 8,
 16,
 9,
 15,
 14,
 13,
 12,
 10,
 11]

In [1]:
import os 

In [2]:
os.path.basename('/hoa/hoctro')

'hoctro'

In [68]:
import pandas as pd
os.environ['dir'] = "/home/xps/educate/code/hust/XQA/data"
from utils.preprocess import string2list, get_label

In [65]:
df = pd.read_csv("/home/xps/educate/code/hust/XQA/data/processed/QA/test.csv")

In [70]:
max = 0
for i in range(len(df)):
    item = df.iloc[i]
    start = string2list(item['start'], type='int')
    text = string2list(item['text'])
    break

In [74]:
text = df['text'][0]

In [78]:
[i.split("'")[1] for i in re.sub('[\[\]]', '' , text).split(',')]

['Chiến tranh thế giới thứ nhất và Cách mạng Đức 1918-1919',
 'Chiến tranh thế giới thứ nhất và Cách mạng Đức 1918-1919',
 'Chiến tranh thế giới thứ nhất và Cách mạng Đức 1918-1919',
 'Chiến tranh thế giới thứ nhất và Cách mạng Đức 1918-1919']